### import

In [37]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")
from datetime import datetime
from datetime import timedelta
import calendar
from dateutil.relativedelta import relativedelta
import functions 

# 출력 옵션
pd.options.display.float_format = '{:,.0f}'.format

def 전회차총분납금계산(row) : 
    # 240830 현재 8회차부터는 모두 예정액 0원이다.
    # 변제예정액3이 0원인 경우 다음회차는 모두 0원이다. 2회차 0원이라도 3회차에는 변제액있는 경우 있음. 이대로 진행해도 될듯
    총분납금 = 0
    if row["변제회차1"] == "" : # 미인가, 변제오류(매입전면책) -> 계산없이 전산 총분납금 리턴
        return row["총분납금"]
    
    for i in range(1,14) :
        구간변제예정액 = int(row[f"변제예정액{i}"])
        if (i >= 3) and (구간변제예정액==0) : # 3회차 이상에서 0원이면 다음 회차도 모두 0원
            break
        구간회차 = row[f"변제회차{i}"]
        
        
        if re.search("~", 구간회차) :
            시작회차 = int(구간회차.split("~")[0])
            종료회차 = int(구간회차.split("~")[1])
            구간회차수 = 종료회차 - 시작회차 + 1
        else : 
            구간회차수 = 1
        구간총분납금 = 구간회차수 * 구간변제예정액
        총분납금 += 구간총분납금
    return 총분납금

### 개인회생 총입금과 총분납금 비교

In [2]:
# 입금파일 읽기
path_depsit = r"C:\Users\DATA\Desktop"
fn_deposit = "카카오종결건_입금조회새창_20240830_1421.xlsx"
deposit = pd.read_excel(join(path_depsit, fn_deposit), dtype=functions.deposit_dtype)

In [3]:
# 개인회생 파일읽기
cutoff_pkl = "240828" # 개인회생새창 pkl 파일 있는 작업일 -> 계좌키는 메인으로 사용할 수 없음
path_rehabilitation = join(r"D:\3.자산\전산 dataset\솔림", cutoff_pkl, "pkl")
fn_rehabilitation = functions.키워드로파일명찾기(path_rehabilitation, "개인회생새창", "기준데이터")
rehabilitation = pd.read_pickle(join(path_rehabilitation, fn_rehabilitation))

In [30]:
# 변제회차열 타입변경 : 변제회차열중에 ~가 없는 경우는 타입이 float라 에러 발생함
rehabilitation.loc[:, rehabilitation.columns.str.contains('변제회차')] = rehabilitation.loc[:, rehabilitation.columns.str.contains('변제회차')].astype(str)
# 전회차 총분납금 계산
rehabilitation["전회차총분납금"] = rehabilitation.apply(lambda row : pd.Series(전회차총분납금계산(row)), axis=1)

In [40]:
rehabilitation[(rehabilitation["전회차총분납금"] != rehabilitation["총분납금"]) & rehabilitation.매각사구분.str.contains("카카오")][["채무자키","분납자성명인","총분납금", "전회차총분납금"]]

,채무자키,분납자성명인,총분납금,전회차총분납금
8247,20525801,강성호,4993138,8811420
9597,20527807,강성호,1800368,3177120
8298,20525866,지연근,9704485,16636260
7844,20525171,김형근,1486290,2286600
8045,20525486,김진철,21071274,44149336
...,...,...,...,...
9084,20527043,황희석,2153099,3370068
9503,20527658,김수현,3500950,4201140
8901,20526766,남가현,15214153,15472020
8039,20525476,강성욱,7592813,9598323
